In [ ]:
#!/usr/bin/env python3

import os
import sys
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time
import logging
import tkinter as tk
from tkinter import ttk, messagebox
from tkinter import filedialog
import queue
import csv
import gspread
from google.oauth2.service_account import Credentials

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    stream=sys.stdout
)

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for each device
gpio_pins_per_device = {
    'Port 1': {"LeftPoke": 17, "RightPoke": 27, "Pellet": 22},
    'Port 2': {"LeftPoke": 10, "RightPoke": 9, "Pellet": 11},
    'Port 3': {"LeftPoke": 0, "RightPoke": 5, "Pellet": 6},
    'Port 4': {"LeftPoke": 13, "RightPoke": 19, "Pellet": 26},
}

# Set all pins as output and initially set them to LOW
for device_pins in gpio_pins_per_device.values():
    for pin in device_pins.values():
        GPIO.setup(pin, GPIO.OUT)
        GPIO.output(pin, GPIO.LOW)

# Splash screen class
class SplashScreen:
    def __init__(self, root, duration=3000):
        self.root = root
        self.root.overrideredirect(True)
        self.root.attributes("-alpha", 1)

        # Get the screen width and height
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()

        # Set the window to be centered
        width = 1300
        height = 450
        x = (screen_width // 2) - (width // 2)
        y = (screen_height // 2) - (height // 2)
        self.root.geometry(f"{width}x{height}+{x}+{y}")

        self.root.configure(bg="black")
        self.root.wm_attributes("-alpha", 1)

        # Create a label for the logo text
        self.label = tk.Label(self.root, text="McCutcheonlab Technologies", font=("Helvetica", 62, "bold"), bg="black", fg="orange")
        self.label.pack(expand=True)

        self.fade_in_out(duration)

    def fade_in_out(self, duration):
        fade_in_time = 1000  # 3 seconds to fade in
        fade_out_time = 2000  # 3 seconds to fade out
        self.fade_in(fade_in_time, lambda: self.fade_out(fade_out_time, self.close_splash))

    def fade_in(self, time_ms, callback):
        alpha = 0.0
        increment = 1 / (time_ms // 50)
        def fade():
            nonlocal alpha
            if alpha < 1.0:
                alpha += increment
                self.root.attributes("-alpha", alpha)
                self.root.after(50, fade)
            else:
                callback()
        fade()

    def fade_out(self, time_ms, callback):
        alpha = 1.0
        decrement = 1 / (time_ms // 50)
        def fade():
            nonlocal alpha
            if alpha > 0.0:
                alpha -= decrement
                self.root.attributes("-alpha", alpha)
                self.root.after(50, fade)
            else:
                callback()
        fade()

    def close_splash(self):
        self.root.destroy()

# Mode selection window
def mode_selection_window(main_app):
    def select_mode(mode):
        main_app.mode = mode
        mode_window.destroy()
        if mode == "RTFED":
            main_app.setup_rtfed_mode()
        elif mode == "TTL":
            main_app.setup_ttl_mode()

    mode_window = tk.Tk()
    mode_window.title("Select Mode")
    
    label = tk.Label(mode_window, text="Please select the mode to run the GUI on:", font=("Helvetica", 12, "bold"))
    label.pack(padx=20, pady=20)

    rtfed_button = tk.Button(mode_window, text="RTFED Mode", font=("Helvetica", 12, "bold"), command=lambda: select_mode("RTFED"))
    rtfed_button.pack(padx=10, pady=10)

    ttl_button = tk.Button(mode_window, text="TTL Mode", font=("Helvetica", 12, "bold"), command=lambda: select_mode("TTL"))
    ttl_button.pack(padx=10, pady=10)

    mode_window.mainloop()

# Main GUI Application Class
class FED3MonitorApp:

    def __init__(self, root):
        self.root = root
        self.root.title("HPFED DATA MONITOR V.01")
        self.mode = None

        self.port_widgets = {}
        self.port_queues = {}
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()
        self.json_path = tk.StringVar()
        self.spreadsheet_id = tk.StringVar()
        self.save_path = ""
        self.data_to_save = {}

        # Google Sheets client setup
        self.gspread_client = None

    def setup_rtfed_mode(self):
        self.clear_gui()

        # Add fields for experimenter and experiment name
        tk.Label(self.root, text="Your Name:", font=("Helvetica", 10, "bold")).grid(column=0, row=2, sticky=tk.E, padx=2, pady=2)
        self.experimenter_entry = ttk.Entry(self.root, textvariable=self.experimenter_name)
        self.experimenter_entry.grid(column=1, row=2, sticky=tk.W, padx=2, pady=2)

        tk.Label(self.root, text="Experiment Name:", font=("Helvetica", 10, "bold")).grid(column=2, row=2, sticky=tk.E, padx=2, pady=2)
        self.experiment_entry = ttk.Entry(self.root, textvariable=self.experiment_name)
        self.experiment_entry.grid(column=3, row=2, sticky=tk.W, padx=2, pady=2)

        # JSON file path for Google Sheets API
        tk.Label(self.root, text="JSON File Path:", font=("Helvetica", 10, "bold")).grid(column=0, row=3, sticky=tk.E, padx=2, pady=2)
        self.json_entry = ttk.Entry(self.root, textvariable=self.json_path)
        self.json_entry.grid(column=1, row=3, sticky=tk.W, padx=2, pady=2)
        self.browse_json_button = tk.Button(self.root, text="Browse JSON File", font=("Helvetica", 10), command=self.browse_json)
        self.browse_json_button.grid(column=2, row=3, padx=5, pady=10)

        # Spreadsheet ID field
        tk.Label(self.root, text="Google Spreadsheet ID:", font=("Helvetica", 10, "bold")).grid(column=0, row=4, sticky=tk.E, padx=2, pady=2)
        self.spreadsheet_entry = ttk.Entry(self.root, textvariable=self.spreadsheet_id)
        self.spreadsheet_entry.grid(column=1, row=4, sticky=tk.W, padx=2, pady=2)

        # Start and Stop buttons
        self.start_button = tk.Button(self.root, text="START RTFED", font=("Helvetica", 15, "bold"), bg="green", fg="white", command=self.start_rtfed_mode)
        self.start_button.grid(column=1, row=5, padx=5, pady=10)
        self.stop_button = tk.Button(self.root, text="STOP(SAVE & QUIT)", font=("Helvetica", 15, "bold"), bg="red", fg="white", command=self.stop_experiment)
        self.stop_button.grid(column=2, row=5, padx=5, pady=10)

    def setup_ttl_mode(self):
        self.clear_gui()

        # Add fields for experimenter and experiment name
        tk.Label(self.root, text="Your Name:", font=("Helvetica", 10, "bold")).grid(column=0, row=2, sticky=tk.E, padx=2, pady=2)
        self.experimenter_entry = ttk.Entry(self.root, textvariable=self.experimenter_name)
        self.experimenter_entry.grid(column=1, row=2, sticky=tk.W, padx=2, pady=2)

        tk.Label(self.root, text="Experiment Name:", font=("Helvetica", 10, "bold")).grid(column=2, row=2, sticky=tk.E, padx=2, pady=2)
        self.experiment_entry = ttk.Entry(self.root, textvariable=self.experiment_name)
        self.experiment_entry.grid(column=3, row=2, sticky=tk.W, padx=2, pady=2)

        # Browse button for selecting data folder
        self.browse_button = tk.Button(self.root, text="Browse Data Folder", font=("Helvetica", 15, "bold"), command=self.browse_folder, bg="gold", fg="blue")
        self.browse_button.grid(column=0, row=3, padx=5, pady=10)

        # Start and Stop buttons
        self.start_button = tk.Button(self.root, text="START TTL", font=("Helvetica", 15, "bold"), bg="green", fg="white", command=self.start_ttl_mode)
        self.start_button.grid(column=1, row=5, padx=5, pady=10)
        self.stop_button = tk.Button(self.root, text="STOP(SAVE & QUIT)", font=("Helvetica", 15, "bold"), bg="red", fg="white", command=self.stop_experiment)
        self.stop_button.grid(column=2, row=5, padx=5, pady=10)

    def clear_gui(self):
        for widget in self.root.winfo_children():
            widget.destroy()

    def browse_json(self):
        self.json_path.set(filedialog.askopenfilename(title="Select JSON File"))

    def browse_folder(self):
        self.save_path = filedialog.askdirectory(title="Select Folder to Save Data")

    def start_rtfed_mode(self):
        # Initialize Google Sheets API with the JSON file
        try:
            creds = Credentials.from_service_account_file(self.json_path.get(), scopes=["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"])
            self.gspread_client = gspread.authorize(creds)
            print(f"Connected to Google Spreadsheet: {self.spreadsheet_id.get()}")
        except Exception as e:
            messagebox.showerror("Error", f"Error setting up Google Sheets API: {e}")
            return

        # Assume data logging happens here, log data to Google Sheets
        print("RTFED Mode: Logging data to Google Sheets")

    def start_ttl_mode(self):
        print("TTL Mode: Sending TTL signals and logging data")
        # Implement the logic to start TTL mode

    def stop_experiment(self):
        print("Stopping experiment...")
        self.save_all_data()

    def save_all_data(self):
        print(f"Saving data in {self.mode} mode")

# Main execution
if __name__ == "__main__":
    splash_root = tk.Tk()
    splash_screen = SplashScreen(splash_root)
    splash_root.after(3000, splash_screen.close_splash)
    splash_root.mainloop()

    root = tk.Tk()
    app = FED3MonitorApp(root)
    mode_selection_window(app)
    root.mainloop()
